In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install ray[train]

In [ ]:
#!pip install -U ipywidgets
# not needed for production, only for jupyter notebook

In [ ]:
#--- need to run 
#memcached -m 10240 -I 20m -vv -u www-data &

In [ ]:
#--- should clean imports - copy paste
import ray
from ray.train import ScalingConfig
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
import ray.train.torch
import ray.util.state
import ray
import torch
import torch.nn as nn
import ray.train.torch
import ray.train
from torch.nn import CrossEntropyLoss, MSELoss
from torch.optim import Adam
from quickannotator.constants import TileStatus
from quickannotator.db import get_session
from quickannotator.db.models import Tile
import sqlalchemy

In [ ]:
ray.init( namespace="quick_annotator")

(RayTrainWorker pid=68566) losses:	 tensor(0.0040, device='cuda:0', grad_fn=<AddBackward0>) tensor(469940., device='cuda:0') tensor(0.0022, device='cuda:0', grad_fn=<MulBackward0>) tensor(0.0018, device='cuda:0', grad_fn=<MulBackward0>)
(RayTrainWorker pid=68566) ntiles 233
(RayTrainWorker pid=68566) <quickannotator.db.models.Tile object at 0x7fbf071f7c70>
(RayTrainWorker pid=68566) 233 1 614 45056 16384 2048 150784 71936
(RayTrainWorker pid=68566) shapes! (2048, 2048, 3) (2048, 2048) (2048, 2048)


In [ ]:
from quickannotator.dl.training import train_pred_loop
# https://discuss.ray.io/t/core-how-to-make-sure-an-actor-is-initialized/1537/5 might be worth keeping 

@ray.remote
class DLActor:
    def __init__(self,actor_name,classid,tile_size,magnification):
        
        print(f"{actor_name=}")
        print(f"{classid=}")
        print(f"{tile_size=}")
        self.actor_name=actor_name #TODO: i don't know if there is a way to get this reflexively instead of passing it in? -- trying to get something working first : ) 
        self.classid=classid
        self.tile_size=tile_size
        self.enable_training=False # starts not training
        self.closedown = False 
        self.allow_pred=True # --- don't know if we'll ever need this, so hard setting to true
        self.hexid=None
        self.magnification = magnification

    def start_dlproc(self,allow_pred=True):
        #--------- this is a bit of a disaster - need to basically make sure we're not launching multple
        #TODO: check if hexid is alive and valid -- can't start two trainings!
        # if is_training(self.hexid):
        #     return False
        #need to check if is already training, if yes, no opt
        print("starting up", self.actor_name)
        #---------------------------

        #scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True,resources_per_worker={"GPU":.5})
        ## will fail on single node single gpu --- (DLActor pid=174521) Duplicate GPU detected : rank 1 and rank 0 both on CUDA device 1000

        #--- NEED A GPU CONTAINER TO TEST THIS        
        total_gpus  = ray.cluster_resources().get("GPU", 0)
        # total_gpus_available=  ray.available_resources().get("GPU", 0)
        # print(total_gpus,total_gpus_available)
        # resource_to_request = (total_gpus_available/total_gpus)/2 * 1.01
        # print(f"resource_to_request: {resource_to_request}")

        scaling_config = ray.train.ScalingConfig(num_workers=int(total_gpus), use_gpu=True,
                                                resources_per_worker={"GPU":.01}, placement_strategy="STRICT_SPREAD")
        
        #----
#        scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)
        trainer = ray.train.torch.TorchTrainer(train_pred_loop,
                                       scaling_config=scaling_config,
                                       train_loop_config={'actor_name':self.actor_name,
                                                          'classid':self.classid,
                                                         'tile_size':self.tile_size,
                                                         'magnification':self.magnification})
        self.hexid = trainer.fit().hex() ##widly -- this doesn't save the result in the actor
        return self.hexid
    
    def infer(self,image_id,tileids): #only bulk should be performed, there is no difference between doing 1 versus 100 tiles, so this reduces code complexity
                                      #NOTE: there is another option here - that we accept ids from the tile table directly instead of having to accept both image_id and tile_id
        if not self.allow_pred:
            print("not doing inference --- actor was started with prediction disabled")
            return False
        # probably need to *Delete* the tiles that are associated with this tileid before executing the below, so that they're not duplicated
        # i would do that in a seperate function as "infer" doesn't logically mean "delete", so the expected behavior might be weird if it delets stuff unprompted
        # perhaps that could be clarified with a e.g., named function paramter - or perhaps a seperate function is really needed
        with get_session() as db_session:
            stmt = sqlalchemy.update(Tile).where(Tile.tile_id.in_(tileids), Tile.image_id == image_id,
                                                Tile.annotation_class_id == self.classid)\
                                                    .values(seen=TileStatus.STARTPROCESSING,date=sqlalchemy.func.now()) ## should add date time

            # Execute the update
            db_session.execute(stmt)

        #need a similar statement to get the DL starting
        return True
    
    def getTileStatus(self,image_id,tile_ids): #probably belongs else where but need this for debug
        with get_session() as db_session:
            stmt = db_session.query(Tile).filter(Tile.tile_id.in_(tile_ids),Tile.image_id == image_id,
                            Tile.annotation_class_id == self.classid)

            result = stmt.all()
        
        return result

    def getClassId(self):
        return self.classid
        
    def getHexId(self):
        return self.hexid

    def setHexId(self,hexid:str): # not sure if set is smart to have -- should likely be done internally
        self.hexid=hexid
        return self.hexid

    def setEnableTraining(self,enable_training: bool):
        self.enable_training=enable_training
        return self.enable_training
    
    def getEnableTraining(self):
        return self.enable_training
    
    def getCloseDown(self):
        return self.closedown
    
    def setCloseDown(self,closedown: bool):
        self.closedown=closedown
        return self.closedown
    

    def getActorName(self):
        return self.actor_name
    
    def getTileSize(self):
        return self.tile_size


this section  is some non-ray testing to see if things work as expected. to use these, remove the @ray annotator in the actor above 

In [ ]:
classid=2
actor_name=f"dlactor_{classid}g"
print(actor_name)
actor = DLActor.options(name=actor_name,lifetime="detached",max_concurrency=2).remote(actor_name=actor_name,classid=classid,
                                                                                      tile_size=2048,magnification=10)


In [ ]:
ray.get(actor.getClassId.remote())

In [ ]:
ray.get(actor.getTileSize.remote())

In [ ]:
ray.get(actor.getActorName.remote()) 

In [ ]:
print(actor_name)

In [ ]:
actor.setEnableTraining.remote(True)

In [ ]:
ref2=actor.start_dlproc.remote()
print(ref2)



In [ ]:
asdfas!!!@3123

In [ ]:
actor.setEnableTraining.remote(True)

In [ ]:
%%timeit
ray.get(actor.getEnableTraining.remote())

In [ ]:
from quickannotator.db import db_session

In [ ]:
##not ray test
class2=DLActor(classid=2)


In [ ]:
class2.infer(image_id=1,tileids=[323],status=1)

In [ ]:
# -- check it was set
tile=class2.getTileStatus([323])
tile[0].seen

In [ ]:
ref2=class2.start_dlproc()


This section is when the @Ray actor is enabled, and thus has remote. note we need max_concurrency = 2 for this to work

In [ ]:
class2 = DLActor.options(max_concurrency=2).remote(classid=2,tile_size=2_048)
ref2=class2.start_dlproc.remote()
print(ref2)

In [ ]:
print(ray.get(class2.getclassid.remote()))


In [ ]:
print(ray.get(class2.infer.remote(image_id=1,tileids=[323],status=1)))

In [ ]:
tile=(ray.get(class2.getTileStatus.remote([323])))
print(tile[0].seen)


In [ ]:
ref2=class2.start_dlproc.remote()
print(ref2)